# Little Lemon analysis and sales report

## importing Required Libraries

In [1]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

## Conecting to MySQL Server

In [2]:
def make_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Server connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

host_name = "localhost"
user_name = "waleed"
user_password = "Waleed@18574" 


connection = make_server_connection(host_name, user_name, user_password)

MySQL Server connection successful


## Use Database

In [3]:
cursor = connection.cursor()
cursor.execute("USE little_lemon_db")

## Task 1: Establish a connection 

In [4]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error
from mysql.connector import PoolError

In [5]:
# Define the database configuration as a dictionary for use in the connection pool.
dbconfig = {
    "database": "little_lemon_db",
    "user": "waleed",
    "password": "Waleed@18574"
}

# Try to create a connection pool named 'pool_b' with 2 connections using the dbconfig 
# settings.

try:
    pool_b = MySQLConnectionPool(pool_name="pool_b", pool_size=2, **dbconfig)
    print("Connection pool 'pool_b' created successfully with 2 connections.")
except Error as err:
    print(f"Error: {err}")

Connection pool 'pool_b' created successfully with 2 connections.


In [13]:
# # Get a connection from the pool and assign it to connection_1
# connection_1 = pool_b.get_connection()
# print("connection_1 acquired from pool_b.")

## Task 2

In [6]:
# Guest booking data
guests = [
    (8, "Anees", "Java", "18:00:00", 6),
    (5, "Bald", "Vin", "19:00:00", 6),
    (12, "Jay", "Kon", "19:30:00", 6)
]

connections = []
cursors = []

# Try to get three connections from the pool and insert bookings
for i, guest in enumerate(guests):
    try:
        connection = pool_b.get_connection()
        connections.append(connection)
        cursor = connection.cursor()
        cursors.append(cursor)
        insert_query = """
            INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
            VALUES (%s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, guest)
        connection.commit()
        print(f"Guest {i+1} booking inserted successfully.")
    except PoolError as pe:
        print(f"PoolError for Guest {i+1}: {pe}")
    except Error as err:
        print(f"Error for Guest {i+1}: {err}")

# Return connections to the pool (only two can be returned)
for i, connection in enumerate(connections):
    try:
        connection.close()
        print(f"Connection {i+1} returned to pool.")
    except PoolError as pe:
        print(f"PoolError when returning connection {i+1}: {pe}")

Guest 1 booking inserted successfully.
Guest 2 booking inserted successfully.
PoolError for Guest 3: Failed getting connection; pool exhausted
Connection 1 returned to pool.
Connection 2 returned to pool.


In [7]:
# Now that the previous two connections have been returned to the pool,
# get a new connection from pool_b and insert the third guest booking.

third_guest = (12, "Jay", "Kon", "19:30:00", 6)

try:
    connection = pool_b.get_connection()
    cursor = connection.cursor()
    insert_query = """
        INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
        VALUES (%s, %s, %s, %s, %s)
    """
    cursor.execute(insert_query, third_guest)
    connection.commit()
    print("Third guest booking inserted successfully.")
    connection.close()
    print("Connection returned to pool.")
except PoolError as pe:
    print(f"PoolError for third guest: {pe}")
except Error as err:
    print(f"Error for third guest: {err}")

Third guest booking inserted successfully.
Connection returned to pool.


In [ ]:
cursor.execute("SELECT * FROM Bookings")
bookings = cursor.fetchall()
bookings_df = pd.DataFrame(bookings, columns=[i[0] for i in cursor.description])
bookings_df

,BookingID,TableNo,GuestFirstName,GuestLastName,BookingSlot,EmployeeID
0,1,12,Anna,Iversen,0 days 19:00:00,1
1,2,12,Joakim,Iversen,0 days 19:00:00,1
2,3,19,Vanessa,McCarthy,0 days 15:00:00,3
3,4,15,Marcos,Romero,0 days 17:30:00,4
4,5,5,Hiroki,Yamane,0 days 18:30:00,2
5,6,8,Diana,Pinto,0 days 20:00:00,5
6,7,8,Anees,Java,0 days 18:00:00,6
7,8,5,Bald,Vin,0 days 19:00:00,6
8,9,12,Jay,Kon,0 days 19:30:00,6


## Task 3

### Name and EmployeeID of the Little Lemon manager

In [8]:
connection = pool_b.get_connection()
cursor = connection.cursor()

# Fetch all records from the Bookings table and display them as a DataFrame

query1 = "SELECT * FROM Employees WHERE Role = 'Manager'"
cursor.execute(query1)
results = cursor.fetchall()

print(cursor.column_names)
for row in results:
    print(row)

('EmployeeID', 'Name', 'Role', 'Address', 'Contact_Number', 'Email', 'Annual_Salary')
(1, 'Mario Gollini', 'Manager', '724, Parsley Lane, Old Town, Chicago, IL', 351258074, 'Mario.g@littlelemon.com', '$70,000')


### Name and Role of the Employee with the Highest Salary

In [9]:
# Query to get the name and role of the employee with the highest salary
query2 = """
SELECT Name, Role
FROM Employees
ORDER BY CAST(REPLACE(REPLACE(Annual_Salary, '$', ''), ',', '') AS UNSIGNED) DESC
LIMIT 1;
"""

cursor.execute(query2)
result = cursor.fetchone()
# print("Name:", result[0])
# print("Role:", result[1])
print(cursor.column_names)
print(result)


('Name', 'Role')
('Mario Gollini', 'Manager')


### Number of Guests Booked Between 18:00 and 20:00

In [10]:
# Query to get the number of guests booked between 18:00 and 20:00
query3 = """
SELECT COUNT(*) AS num_guests
FROM Bookings
WHERE BookingSlot BETWEEN '18:00:00' AND '20:00:00';
"""

cursor.execute(query3)
result = cursor.fetchone()
print("Number of guests booked between 18:00 and 20:00:", result[0], "guests")

Number of guests booked between 18:00 and 20:00: 7 guests


## Full Name and BookingID of all guests waiting to be Seated

In [11]:
# Query to get the full name and BookingID of all guests waiting to be seated with the receptionist,
# sorted by their BookingSlot

query4 = """
SELECT 
    CONCAT(GuestFirstName, ' ', GuestLastName) AS full_name,
    BookingID
FROM Bookings
WHERE EmployeeID = (
    SELECT EmployeeID FROM Employees WHERE Role = 'Receptionist'
)
ORDER BY BookingSlot;
"""

cursor.execute(query4)
results = cursor.fetchall()
# print("Full Name | BookingID")
# for row in results:
#     print(row[0], "|", row[1])
print(cursor.column_names)
for row in results:
    print(row)

('full_name', 'BookingID')
('Anees Java', 7)
('Bald Vin', 8)
('Jay Kon', 9)


## Task 4

In [12]:
# Create the BasicSalesReport stored procedure
basic_sales_report_proc = """
CREATE PROCEDURE BasicSalesReport()
BEGIN
    SELECT
        SUM(BillAmount) AS total_sales,
        AVG(BillAmount) AS average_sale,
        MIN(BillAmount) AS minimum_bill_paid,
        MAX(BillAmount) AS maximum_bill_paid
    FROM Orders;
END
"""


# Drop the procedure if it exists
cursor.execute("DROP PROCEDURE IF EXISTS BasicSalesReport")

# Create the procedure
cursor.execute(basic_sales_report_proc)

# Call the stored procedure
cursor.callproc('BasicSalesReport')

# # Fetch and print results
# for result in cursor.stored_results():
#     dataset = result.fetchall()
#     columns = [desc[0] for desc in result.description]
#     print(columns)
#     for row in dataset:
#         print(row)


# Fetch and print results as a DataFrame
for result in cursor.stored_results():
    dataset = result.fetchall()
    columns = [desc[0] for desc in result.description]
    df = pd.DataFrame(dataset, columns=columns)
    print(df)



  total_sales average_sale  minimum_bill_paid  maximum_bill_paid
0         243      48.6000                 37                 86


/tmp/ipykernel_3137/4113673372.py:34: DeprecationWarning: Call to deprecated function stored_results. Reason: The property counterpart 'stored_results' will be added in a future release, and this method will be removed.
  for result in cursor.stored_results():


In [24]:
cursor.close()
connection.close()

## Task 5

In [25]:
connection = pool_b.get_connection()

cursor = connection.cursor(buffered = True)

In [26]:
# Query to get the next three upcoming bookings with guest and assigned employee info
query = """
SELECT 
    b.BookingSlot,
    CONCAT(b.GuestFirstName, ' ', b.GuestLastName) AS Guest_name,
    CONCAT('Assigned to: ', e.Name, ' [', e.Role, ']') AS Assigned_to
FROM Bookings b
LEFT JOIN Employees e ON b.EmployeeID = e.EmployeeID
ORDER BY b.BookingSlot ASC
LIMIT 3;
"""

cursor.execute(query)
results = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
upcoming_df = pd.DataFrame(results, columns=columns)
print(upcoming_df)

# Return the connection to the pool
cursor.close()
connection.close()

      BookingSlot        Guest_name                               Assigned_to
0 0 days 15:00:00  Vanessa McCarthy  Assigned to: Giorgos Dioudis [Head Chef]
1 0 days 17:30:00     Marcos Romero  Assigned to: Fatma Kaya [Assistant Chef]
2 0 days 18:00:00        Anees Java   Assigned to: John Millar [Receptionist]
